# Sorting the conferences via "date" field

In [ ]:
import datetime, json
import pandas as pd

## Sorting with pandas

In [ ]:
confs = pd.read_json("_data/Conference.json")
confs.dropna(inplace=True)
confs

In [ ]:
import pprint

In [ ]:
pprint.pprint(
    confs.to_json(
        orient="records",
        date_format="iso",
    )
)

In [ ]:
result = confs.sort_values("date", ascending=False).to_json(
    orient="records", date_format="iso"
)
parsed = json.loads(result)
json.dump(parsed, open("_data/sorted_confs.json", "w"), indent=4)

## Creating the bibfile from inspire

In [ ]:
import requests

url = "https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=a Guillermo.Fidalgo.1&format=bibtex"
inspire = requests.get(url).text
# response = requests.get('https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=Guillermo%20Fidalgo')
# url = response.json()['links']['bibtex']
# print(url)
# pprint.pprint(json_out['links'])
# inspire = requests.get(url).text
print(inspire)

In [ ]:
zenodo = r"""
@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}
"""
print(zenodo)

Making it into a script

In [ ]:
with open("bibfile.bib", "w") as f:
    f.write(inspire)
    f.write(zenodo)

In [ ]:
%%writefile createbib.py 
import requests
response = requests.get('https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=Guillermo%20Fidalgo')
url = response.json()['links']['bibtex']
inspire = requests.get(url).text

zenodo = r"""
@misc{alexander_moreno_briceno_2022_7115834,
  author       = {Alexander Moreno Briceño and Aman Goel and Guillermo Antonio Fidalgo Rodriguez},
  title        = {Teaching Python the Sustainable Way: Lessons Learned at HSF Training},
  month        = sep,
  year         = 2022,
  publisher    = {Zenodo},
  doi          = {10.5281/zenodo.7115834},
  url          = {https://doi.org/10.5281/zenodo.7115834}
}
"""
with open("_data/bibfile.bib",'w') as f:
    f.write(inspire)
    f.write(zenodo)

In [ ]:
! python createbib.py

## Making tests with Pandoc

Just run
```shell
pandoc -C _data/Publications.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/myPubs.md
```

In [ ]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
-o _includes/myPubs.md

In [ ]:
! pandoc -C bibfile.bib -t markdown_strict \
--csl american-physics-society.csl \
-o deleteME.md

In [ ]:
!pandoc --help